In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
code= """

#include<stdio.h>
#include<omp.h>
#include<stdlib.h>
#include<time.h>

void mergesort(int *a,int i,int j);
void mergesortserial(int *a,int i,int j);
void merge(int *a,int i1,int j1,int i2,int j2);

int main()
{
    int *a,num;
    
    scanf("%d",&num);
    
    a = (int *)malloc(sizeof(int)*num);
    
    for(int i=0;i<num;i++)
    {
      a[i]=rand()%num+1;
    }
    
    double start,end;
    start = omp_get_wtime();
    mergesortserial(a,0,num-1);
    end = omp_get_wtime();
    
    printf("Time serial: %lf",end-start);
    
    start = omp_get_wtime();
    mergesort(a,0,num-1);
    end = omp_get_wtime();
    
    printf("Time parallel :%lf",end-start);
    
    for(int i=0;i<num;i++)
    {
      printf("%d",a[i]);
    }

}

void mergesort(int *a,int i,int j)
{
   int mid;
   
   if(i<j)
   {
      mid=(i+j)/2;
      
      #pragma omp parallel sections
      {
        #pragma omp section
        {
            mergesort(a,0,mid);
        }
        #pragma omp section
        {
            mergesort(a,mid+1,j);
        }
      }
      merge(a,0,mid,mid+1,j);
   }
}

void mergesortserial(int *a,int i,int j)
{
   int mid;
   
   if(i<j)
   {
      mid=(i+j)/2;
      
      mergesort(a,0,mid);
        
      mergesort(a,mid+1,j);
        
      merge(a,0,mid,mid+1,j);
   }
}

void merge(int *a,int i1,int j1,int i2,int j2)
{
    int arr[1000];
    
    int i=i1,j=i2;
    int k=0;
    
    while(i<=j1 && j<=j2)
    {
      if(a[i]<a[j])
        arr[k++]=a[i++];
      else
        arr[k++]=a[j++];
    }
    
    while(i<=j1)
      arr[k++]=a[i++];
      
    while(j<=j2)
      arr[k++]=a[j++];
      
    for(i=i1;i<=j2;i++)
    {
      a[i]=arr[i];
    }
}
"""

In [0]:
text_file = open('sort.c',"w")
text_file.write(code)
text_file.close()

In [0]:
!gcc -o sort -fopenmp sort.c

In [0]:
!./sort

5
Time serial: 0.000211Time parallel :0.00004012344

In [0]:
code1="""

#include<stdio.h>
#include<stdlib.h>
#include<omp.h>
#include<time.h>

void bubblesort(int *a,int n);
void simplebubble(int *a,int n);

int main()
{
    int *a,n,*b;
    
    scanf("%d",&n);
    
    a=(int *)malloc(sizeof(int)*n);
    b=(int *)malloc(sizeof(int)*n);
    for(int i=0;i<n;i++)
    {
      a[i]=rand()%n+1;
      b[i]=a[i];
    }
      
      
    double start,end;
    start = omp_get_wtime();
    simplebubble(a,n);
    end = omp_get_wtime();
     
    printf("%lf",end-start);
    
    start = omp_get_wtime();
    bubblesort(b,n);
    end = omp_get_wtime();
    
    
    printf("%lf",end-start);
    for(int i=0;i<n;i++)
      printf("%d",a[i]);
      
}

void simplebubble(int *a, int n)
{
    for(int i=0;i<n-1;i++)
    {
        for(int j=0;j<n-1-i;j++)
        {
            if(a[j]>a[j+1])
            {
                int temp=a[j+1];
                a[j+1]=a[j];
                a[j]=temp;
            }
        }
   }     
}


void bubblesort(int *a,int n)
{

  for(int i=0;i<n;i++)
  {
      if(i%2==0)
      {
              #pragma omp parallel for
              for(int j=0;j<n;j+=2)
              {
                  if(j+1<n && a[j]>a[j+1])
                  {
                      int temp=a[j];
                      a[j]=a[j+1];
                      a[j+1]=temp;
                  }
              }
          
      }
      else
      {
              #pragma omp parallel for
              for(int j=1;j<n-1;j+=2)
              {
                  if(j+1<n && a[j]>a[j+1])
                  {
                      int temp=a[j];
                      a[j]=a[j+1];
                      a[j+1]=temp;
                  }
              }
          
      }
  }
}



"""

In [0]:
f1 = open('sort1.c','w')
f1.write(code1)
f1.close()

In [0]:
!gcc -o sort1 -fopenmp sort1.c

In [0]:
!./sort1

In [0]:
#cpp

In [0]:
codeb="""

#include<bits/stdc++.h>
#define SIZE 2000
using namespace std;

void bubblesort(int *a)
{
	for(int i=0;i<SIZE;i++)
	{
		for(int j=0;j<SIZE-1;j++)
		{
			if(a[j]>a[j+1])
			{
				swap(a[j],a[j+1]);
			}
		}
	}
}

void parallel_bubblesort(int *a)
{
	for(int i=0;i<SIZE;i++)
	{
		int first = i%2;

		#pragma omp parallel for default(none),shared(a,first)
		for(int j=first;j<SIZE-1;j=j+1)
		{
			if(a[j] > a[j+1]){
				swap(a[j], a[j+1]);
			}
		}
	}
}

int main()
{
	int *a,*b;
	a = (int *)malloc(SIZE*sizeof(int));
	b = (int *)malloc(SIZE*sizeof(int));
	
	for(int i=0;i<SIZE;i++)
	{
		a[i]=rand()%324392;
		b[i]=a[i];
		
	}

	clock_t timer = clock();
	bubblesort(a);
	cout<<"Time taken for serial bubble sort"<<(float)(clock() - timer)/CLOCKS_PER_SEC;
	cout<<"\n";
	timer = clock();
	parallel_bubblesort(b);
	cout<<"Time taken for parallel bubble sort"<<(float)(clock()-timer)/CLOCKS_PER_SEC;
	for(int i = 0;i < SIZE;i ++){
		if(a[i] != b[i]){
			cout << "\nIncorrect result";
			break;
		}
	}

}
"""

In [0]:
fz = open('sortb.cpp','w')
fz.write(codeb)
fz.close()

In [0]:
!g++ -fopenmp sortb.cpp